In [2]:
from gbdxtools import Interface
from gbdxtools import CatalogImage
from gbdxtools.task import env
import json

gbdx = Interface()

In [3]:
catalog_id = env.inputs.get('catalog_id', '1030010012325400')
image = CatalogImage(catalog_id)
imgBounds = image.bounds
print(imgBounds)

(-8.68061063, 7.00726822431908, -8.517729332838664, 7.55223116)


In [5]:
metadata = image.ipe.metadata['image']
#print(json.dumps(metadata, indent=4))

In [6]:
data_url = 's3://receiving-dgcs-tdgplatform-com/057792516010_01_003'

In [9]:
ao_task = gbdx.Task('AOP_Strip_Processor',
                    data=data_url,
                    bands='MS',
                    dra_mode='BaseLayerMatch')
ao_task.impersonation_allowed = True

In [10]:
aoiBounds = [-8.703095, 7.006610, -8.499084, 7.276814]

print('image bounds', imgBounds)
print('chose bounds', aoiBounds)

bounds = [max(imgBounds[0], aoiBounds[0]), max(imgBounds[1], aoiBounds[1]),
                min(imgBounds[2], aoiBounds[2]), min(imgBounds[3], aoiBounds[3])]

print('fixed bounds', bounds)

('image bounds', (-8.68061063, 7.00726822431908, -8.517729332838664, 7.55223116))
('chose bounds', [-8.703095, 7.00661, -8.499084, 7.276814])
('fixed bounds', [-8.68061063, 7.00726822431908, -8.517729332838664, 7.276814])


In [11]:
# problem: ship value may not be set
clip_task = gbdx.Task('gdalcrop',                      
                     #image='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/lt_ao_bs_match1/',
                     image=ao_task.outputs.data.value,
                     crop='{} {} {} {}'.format(bounds[0], bounds[1], bounds[2], bounds[3]),
                     ship="false")

In [12]:
tile_task = gbdx.Task('splitter-gbdx:0.0.5',
                     data_in=clip_task.outputs.cropped.value)

In [13]:
fe_task = gbdx.Task('fe-gbdx:0.0.9',
                   data_in=tile_task.outputs.data_out.value)

In [15]:
clustervis_task = gbdx.Task('clustervis-gbdx:0.0.13',
                           data_in=tile_task.outputs.data_out.value,
                           features_in=fe_task.outputs.data_out.value,
                           #data_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/tiles2',
                           #features_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/fe-out2',
                           train='true')

In [18]:
#tasks = [ao_task]
#tasks2 = [clip_task]
#tasks = [ao_task, clip_task]
#tasks = [clip_task, tile_task]
#tasks = [fe_task]
#tasks = [clustervis_task]
#tasks = [clip_task, tile_task, fe_task, clustervis_task]
tasks = [ao_task, clip_task, tile_task, fe_task, clustervis_task]
workflow = gbdx.Workflow(tasks)

In [19]:
workflow.savedata(ao_task.outputs.data, location='liberia-train-acomp')
workflow.savedata(clip_task.outputs.cropped, location='liberia-train-clip')
workflow.savedata(tile_task.outputs.data_out, location='liberia-train-tiles')
workflow.savedata(fe_task.outputs.data_out, location='liberia-train-fe')
workflow.savedata(clustervis_task.outputs.data_out, location='liberia-train-clustervis-out')

In [20]:
workflow.execute()

u'4836959582321433586'

In [33]:
print workflow.id
print workflow.status
#print(json.dumps(workflow.events[len(workflow.events)-1], indent=4))
#print(json.dumps(workflow.events, indent=4))
for e in workflow.events:
    print("    {} {}".format(e['task'], e['event']))
    #print(e)
print(json.dumps(workflow.stdout, indent=4))
print(json.dumps(workflow.stderr, indent=4))

4836959582321433586
{u'state': u'complete', u'event': u'succeeded'}
    AOP_Strip_Processor_5a8a5e20 submitted
    gdalcrop_517a8349 submitted
    splitter-gbdx_0.0.5_86d504f0 submitted
    fe-gbdx_0.0.9_d60fda2c submitted
    clustervis-gbdx_0.0.13_402a75fa submitted
    AOP_Strip_Processor_5a8a5e20 scheduled
    AOP_Strip_Processor_5a8a5e20 started
    AOP_Strip_Processor_5a8a5e20 succeeded
    gdalcrop_517a8349 scheduled
    gdalcrop_517a8349 started
    gdalcrop_517a8349 succeeded
    splitter-gbdx_0.0.5_86d504f0 scheduled
    splitter-gbdx_0.0.5_86d504f0 started
    splitter-gbdx_0.0.5_86d504f0 succeeded
    fe-gbdx_0.0.9_d60fda2c scheduled
    fe-gbdx_0.0.9_d60fda2c started
    fe-gbdx_0.0.9_d60fda2c succeeded
    clustervis-gbdx_0.0.13_402a75fa scheduled
    clustervis-gbdx_0.0.13_402a75fa started
    clustervis-gbdx_0.0.13_402a75fa succeeded
[
    {
        "stdout": "Tiles Input dir: /mnt/work/input/data_in\nFeatures Input dir: /mnt/work/input/data_in\nTrain: True\nOutput dir:

[
    {
        "stderr": "/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.\n  \"This module will be removed in 0.20.\", DeprecationWarning)\n/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.\n  warnings.warn(msg, category=DeprecationWarning)\n", 
        "id": "4836959582320642201", 
        "name": "clustervis-gbdx_0.0.13_402a75fa", 
        "taskType": "clustervis-gbdx:0.0.13"
    }, 
    {
        "s

In [ ]:
#workflow.cancel()